# ETL Pipeline

This notebook is executed using Databricks Workflows as defined in  resources/bundle_assets_job.yml.


In [ ]:
from lib.get_news import get_news
from lib.transforms import clean_news, sentiment_news
from pyspark.sql import SparkSession

In [ ]:
# Create a Spark session 
spark = SparkSession.builder.getOrCreate()

## ingest data

In [ ]:
# Get news data
tickers = ["AAPL", "MSFT", "SNOW", "GOOGL", "TSLA"]
bronze_path = "/mnt/news/bronze/"
get_news(tickers, bronze_path)

## transform Data

In [ ]:
# Clean the news data
spark.sql(
    "CREATE TABLE IF NOT EXISTS news_bronze USING DELTA LOCATION '/mnt/news/bronze'"
)
input_table = "news_bronze"  # Define the input table name
clean_df = clean_news(input_table)

## save to path

In [ ]:
# Perform sentiment analysis on the cleaned data
mode = "append"
silver_path = "/mnt/news/silver/"
output_table = "news_silver"  # Define the output table name
sentiment_news(clean_df, mode, silver_path, output_table)